In [1]:
from mesa import Agent, Model
from mesa.space import NetworkGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import networkx as nx
import graph_init
import pandas as pd
import station as st

import network_example as ne
import numpy as np
import pathfinding as pf

In [61]:
class Commuter(Agent):
    def __init__(self, model, current_pos, distance_left, intermediate_node, destination):
        super().__init__(model)
        self.destination = destination
        self.intermediate_node = intermediate_node
        self.distance_left = distance_left
        self.current_pos = current_pos
        self.biking = False

        assert self.model.grid.G.nodes[self.current_pos]['type'] == 'destination', "Agent must start at a destination"
        
    def step(self):
        self.current_pos = self.model.get_node(self)  # Update the agent's current position
        if self.distance_left <= 0:
            # If the agent has reached the intermediate node, set new start node
            self.current_pos = self.intermediate_node
            self.model.grid.move_agent(self, self.current_pos)

            # Return bike if the agent is at the intended station and there are spots available
            if self.biking and self.model.grid.G.nodes[self.current_pos]['type'] == 'station' and self.model.grid.G.nodes[self.current_pos]['data'].get_spot_availability():
                self.model.grid.G.nodes[self.current_pos]['data'].return_bike()
                self.biking = False

            # If the agent has reached the destination, set new destination
            # Else, the agent will rent a bike if it is at a station and there are bikes available
            if self.current_pos == self.destination:
                self.destination = self.model.sample_destination(self.current_pos)
            elif not(self.biking) and self.model.grid.G.nodes[self.current_pos]['type'] == 'station' and self.model.grid.G.nodes[self.current_pos]['data'].get_bike_availability():
                self.model.grid.G.nodes[self.current_pos]['data'].rent_bike()
                self.biking = True

            # Set new intermediate node
            if self.model.grid.G.nodes[self.current_pos]['type'] == 'destination':
                path = pf.pathfind(self.model.grid.G, self.current_pos, self.destination, False)
            else:
                bike = self.model.grid.G.nodes[self.current_pos]['data'].get_bike_availability()
                path = pf.pathfind(self.model.grid.G, self.current_pos, self.destination, bike)

            self.intermediate_node = path[0]

            if path[1]:
                self.distance_left = self.model.grid.G[self.current_pos][self.intermediate_node]['weight']
            else:
                self.distance_left = self.model.grid.G[self.current_pos][self.intermediate_node]['weight'] * self.model.walking_multiplier
        else:
            self.distance_left -= 1        

    
    def get_agent_position(self):
    # needed seperate function for data collector
        return self.current_pos
    
    def get_distance_left(self):
    # needed seperate function for data collector
        return self.distance_left
    
    def get_agent_destination(self):
        return self.destination
    
    def get_intermediate_node(self):
        return self.intermediate_node


class MyModel(Model):
    def __init__(self, n_agents):
        super().__init__()
        g = ne.basic_graph()[0]
        self.walking_multiplier = 3
        self.grid = NetworkGrid(g)
        # self.schedule = RandomActivation(self)
        self.datacollector = DataCollector(
            agent_reporters={"Position": lambda agent: agent.get_agent_position(),
                             "Distance_Left": lambda agent: agent.get_distance_left(),
                             "Intermediate_Node": lambda agent: agent.get_intermediate_node(),
                             "Destination": lambda agent: agent.get_agent_destination()})
        
        self.stations = pf.get_stations(self.grid.G)
        self.destinations = pf.get_destinations(self.grid.G)
        self.destination_w = ne.basic_weights()

        # Create agents and place them on the grid
        for i in range(1, n_agents + 1):
            node_id = self.random.choice(self.destinations)

            destination_node = self.sample_destination(node_id)
            path = pf.pathfind(self.grid.G, node_id, destination_node, False)
            intermediate_node = path[0]
            distance_left = self.grid.G[node_id][intermediate_node]['weight'] * self.walking_multiplier

            commuter = Commuter(self, current_pos=node_id, distance_left=distance_left, intermediate_node=intermediate_node, destination=destination_node)
            # self.schedule.add(commuter)
            self.grid.place_agent(commuter, node_id)
            
        self.datacollector.collect(self)
    
    def get_node(self, agent):
        # Iterate over all node IDs in the graph
        for node_id in self.grid.G.nodes():
            # Get the agents at the current node
            agents_at_node = self.grid.get_cell_list_contents([node_id])
            # If the given agent is found at this node, return the node ID
            if agent in agents_at_node:
                return node_id
        return None  # If the agent is not found in any node
    
    def sample_destination(self, current_pos):
        while True:
            destination = np.random.choice(self.destinations, p=ne.basic_weights())
            if destination != current_pos:
                return destination

    def step(self):
        # print("Model step")
        # self.schedule.step()
        self.agents.shuffle_do("step")
        self.datacollector.collect(self)

In [76]:
model = MyModel(1)
for i in range(100):
    print(model.agents[0].get_agent_position(), model.agents[0].get_distance_left(), model.agents[0].get_agent_destination(), model.agents[0].get_intermediate_node())
    model.step()

agent_data = model.datacollector.get_agent_vars_dataframe()
agent_id = 8
agent_specific_data = agent_data[agent_data.index.get_level_values('AgentID') == agent_id]

# Print the filtered DataFrame
# print(agent_specific_data)

6 36.0 7 7
6 35.0 7 7
6 34.0 7 7
6 33.0 7 7
6 32.0 7 7
6 31.0 7 7
6 30.0 7 7
6 29.0 7 7
6 28.0 7 7
6 27.0 7 7
6 26.0 7 7
6 25.0 7 7
6 24.0 7 7
6 23.0 7 7
6 22.0 7 7
6 21.0 7 7
6 20.0 7 7
6 19.0 7 7
6 18.0 7 7
6 17.0 7 7
6 16.0 7 7
6 15.0 7 7
6 14.0 7 7
6 13.0 7 7
6 12.0 7 7
6 11.0 7 7
6 10.0 7 7
6 9.0 7 7
6 8.0 7 7
6 7.0 7 7
6 6.0 7 7
6 5.0 7 7
6 4.0 7 7
6 3.0 7 7
6 2.0 7 7
6 1.0 7 7
6 0.0 7 7
7 5.0 5 3
7 4.0 5 3
7 3.0 5 3
7 2.0 5 3
7 1.0 5 3
7 0.0 5 3
3 10.0 5 1
3 9.0 5 1
3 8.0 5 1
3 7.0 5 1
3 6.0 5 1
3 5.0 5 1
3 4.0 5 1
3 3.0 5 1
3 2.0 5 1
3 1.0 5 1
3 0.0 5 1
1 15.0 5 5
1 14.0 5 5
1 13.0 5 5
1 12.0 5 5
1 11.0 5 5
1 10.0 5 5
1 9.0 5 5
1 8.0 5 5
1 7.0 5 5
1 6.0 5 5
1 5.0 5 5
1 4.0 5 5
1 3.0 5 5
1 2.0 5 5
1 1.0 5 5
1 0.0 5 5
5 5.0 7 1
5 4.0 7 1
5 3.0 7 1
5 2.0 7 1
5 1.0 7 1
5 0.0 7 1
1 10.0 7 3
1 9.0 7 3
1 8.0 7 3
1 7.0 7 3
1 6.0 7 3
1 5.0 7 3
1 4.0 7 3
1 3.0 7 3
1 2.0 7 3
1 1.0 7 3
1 0.0 7 3
3 15.0 7 7
3 14.0 7 7
3 13.0 7 7
3 12.0 7 7
3 11.0 7 7
3 10.0 7 7
3 9.0 7 7
3 8.0 7 7
3 7.0 7 7

(0, 0.0, 6, 1)